# Оценка точности модели

В этой секции тестируется модель YOLO, обученная для детекции монет. Мы проводим валидацию модели на тестовом наборе данных и выводим метрики производительности.

In [1]:
# Импорт необходимых библиотек
from ultralytics import YOLO
import torch

# Проверяем, доступен ли CUDA для ускорения вычислений на GPU
torch.cuda.is_available()

True

In [2]:
model = YOLO('CoinCounter/models/model.onnx', task='detect')
data_yaml = 'datasets/data.yaml' 

# Валидация модели на тестовом наборе данных
results  = model.val(data=data_yaml, split='test',verbose=False)

Ultralytics YOLOv8.2.91  Python-3.12.4 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
Loading CoinCounter\models\model.onnx for ONNX Runtime inference...
Setting batch=1 input of shape (1, 3, 640, 640)


val: Scanning A:\rep\portfolio\detetct\CoinCounter\datasets\test\labels.cache... 131 images, 45 backgrounds, 0 corrupt: 100%|██████████| 131/131 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 29, len(boxes) = 683. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 131/131 [00:01<00:00, 70.06it/s]


                   all        131        683      0.928      0.827      0.888       0.75
Speed: 0.4ms preprocess, 9.7ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to a:\rep\portfolio\detetct\CoinCounter\runs\detect\val


In [3]:
# Выводим метрики тестирования модели
# Precision, Recall, mAP@0.5, mAP@0.5:0.95
print(f"Precision: {results.results_dict['metrics/precision(B)']}")
print(f"Recall: {results.results_dict['metrics/recall(B)']}")
print(f"mAP@0.5: {results.results_dict['metrics/mAP50(B)']}")
print(f"mAP@0.5:0.95: {results.results_dict['metrics/mAP50-95(B)']}")

Precision: 0.9275618276560502
Recall: 0.827353813454718
mAP@0.5: 0.888304043578894
mAP@0.5:0.95: 0.7499277467515681


## Описание метрик

1. **Precision (Точность)** — 0.9276:
   - Доля правильных предсказаний модели среди всех сделанных ею предсказаний. Если модель сообщает, что нашла объект, то с вероятностью 92.76% это действительно правильное определение.

2. **Recall (Полнота)** — 0.8273:
   - Доля правильных предсказаний модели среди всех объектов, которые действительно существуют в данных. Полнота показывает, насколько хорошо модель находит все истинные объекты.

3. **mAP@0.5** — 0.8886:
   - **Mean Average Precision (Средняя точность)** при пороге 0.5 (IoU ≥ 0.5). Это среднее значение точности по всем классам при условии, что предсказанный bounding box и истинный пересекаются не менее чем на 50%.

4. **mAP@0.5:0.95** — 0.7501:
   - Средняя точность при различных порогах пересечения IoU от 0.5 до 0.95 с шагом 0.05. Этот показатель учитывает различные уровни перекрытия объектов и предсказанных рамок.